In [1]:
import pandas as pd
import os
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from pathlib import Path
import glob

In [2]:
input_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Target/Raw/Nov/"
output_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Result/Payout Status/"
output_loc_invalid = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Result/Payout Status/Invalid/"
output_loc_final = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Result/Payout Status/Final/"

In [3]:
First_ingestion_of_month = "2022/11/04"

In [4]:
current_date = ["Nov-28-2022"]

In [5]:
default_ingestion_date = "2022/06/08"

In [6]:
current_month = "Nov"
current_year = 2022

In [7]:
date_1 = range(1,10)
date_2 = range(1,17)

In [8]:
Date = []
for i in date_1:
    i = str(current_month)+"-"+str(0)+str(i)+"-"+str(current_year)
    Date.append(i)

for i in date_2:
    i = str(current_month)+"-"+str(i)+"-"+str(current_year)
    Date.append(i)

# Google File

In [9]:
input_loc_g = input_loc+"Google/Reconsilation Final/"

In [10]:
filelist = glob.glob(input_loc_g+"**/*.csv", recursive = True)

In [11]:
filelist

['/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Target/Raw/Nov/Google/Reconsilation Final/Google_Reconsilation_status_17_to_Remaining.csv',
 '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Target/Raw/Nov/Google/Reconsilation Final/Google_Reconsilation_status_01_to_16.csv']

In [12]:
Google_re = pd.DataFrame()

for file in filelist:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
        Google_re = Google_re.append(df)

In [13]:
Google_re.sort_values(by=['sno'], inplace=True)

In [14]:
Google_re['sno'] = Google_re['sno'].astype(str)

In [15]:
Google_re.shape

(174874, 34)

In [16]:
Google_re.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Is in Previous Month Payout,Is in Previous Month Payout Date,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag
0,1,Akola,Akola,7196591.0,8381097116,43260,14549.0,BCR2DN4T2C2KVQ2C,VALID,NaN,...,No Match,NaN,NaN,NaN,1,1,No,No,NaN,Google


In [17]:
Google_re.sort_values(by=['Validity', 'Date'], ascending = [False, True], inplace=True)
len(Google_re)

174874

# Invalid_IncompleteSubTasks

In [18]:
Google_re_f = Google_re[['sno', 'MerchantExternalId', 'StorePhoneNumber', 'Task Count', 'Validity', 'InvalidReasons']]

In [19]:
Google_re_f2 = Google_re_f[['sno','Validity', 'InvalidReasons' , 'Task Count']]

In [20]:
Google_re_f3 = Google_re_f2.loc[(Google_re_f2['Task Count']<3)]

In [21]:
Google_re_f3['Payout_Status_1'] = 'Invalid_IncompleteSubTasks'

In [22]:
Google_re_f4 = Google_re_f3[Google_re_f3['InvalidReasons'].str.contains('Invalid_PhonenumberLookup', na=False)]

In [23]:
Google_re_f4['Payout_Status_2'] = 'Invalid_PhonenumberLookup'

In [24]:
Google_re_f4_f = Google_re_f4[['sno', 'Payout_Status_2']]

In [25]:
Google_re_f3_merge = Google_re_f3.merge(Google_re_f4_f, on='sno', how='left')

In [26]:
Google_re_f3_merge.drop_duplicates(subset=['sno'], inplace=True)

In [27]:
Google_re_f3_merge['New_Payout_Status_1'] = np.nan
Google_re_f3_merge['New_Payout_Status_1'].fillna(Google_re_f3_merge['Payout_Status_2'], inplace=True)
Google_re_f3_merge['New_Payout_Status_1'].fillna(Google_re_f3_merge['Payout_Status_1'], inplace=True)

In [28]:
Google_re_f3_merge_f = Google_re_f3_merge[['sno', 'New_Payout_Status_1']]

In [29]:
Google_re_f3_merge_f.head(1)

,sno,New_Payout_Status_1
0,45,Invalid_IncompleteSubTasks


In [30]:
Google_re_1 = Google_re.merge(Google_re_f3_merge_f, how='left', on='sno')
Google_re_1.drop_duplicates(subset=['sno'], inplace=True)

# Duplicate Merchant External ID

In [31]:
Google_re_1_mid = Google_re_1[['sno', 'Ext_ID Dup?']]
Google_re_1_mn = Google_re_1[['sno', 'Phone_Dup?']]

In [32]:
Google_re_1_mid_f = Google_re_1_mid.loc[(Google_re_1_mid['Ext_ID Dup?']>1)]

In [33]:
Google_re_1_mid_f['Payout_Status_1'] = "Duplicate Ext_ID"

In [34]:
Google_re_1_mid_f2 = Google_re_1_mid_f[['sno', 'Payout_Status_1']]

In [35]:
Google_re_1_mn_f = Google_re_1_mn.loc[(Google_re_1_mn['Phone_Dup?']>1)]

In [36]:
Google_re_1_mn_f['Payout_Status_2'] = "Duplicate Phone_No"

In [37]:
Google_re_1_mn_f2 = Google_re_1_mn_f[['sno', 'Payout_Status_2']]

In [38]:
Google_re_1_merge = Google_re_1.merge(Google_re_1_mid_f2, on='sno', how='left')
Google_re_1_merge.drop_duplicates(subset=['sno'], inplace=True)

In [39]:
Google_re_1_merge2 = Google_re_1_merge.merge(Google_re_1_mn_f2, on='sno', how='left')
Google_re_1_merge2.drop_duplicates(subset=['sno'], inplace=True)

In [40]:
Google_re_2 = Google_re_1_merge2

In [41]:
Google_re_2['New_Payout_Status_2'] = np.nan
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['Payout_Status_1'], inplace=True)
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['Payout_Status_2'], inplace=True)
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['New_Payout_Status_1'], inplace=True)
Google_re_2.drop(['Payout_Status_1', 'Payout_Status_2', 'New_Payout_Status_1'], axis=1, inplace=True)

In [42]:
Google_re_2.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Is in Previous Month Payout Date,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2
0,1,Akola,Akola,7196591.0,8381097116,43260,14549.0,BCR2DN4T2C2KVQ2C,VALID,NaN,...,NaN,NaN,NaN,1,1,No,No,NaN,Google,NaN


# Valid

In [43]:
Google_re_2_f = Google_re_2[['sno', 'Validity']]

In [44]:
Google_re_2_f['Validity'].value_counts()

VALID      157524
INVALID     17350
Name: Validity, dtype: int64

In [45]:
Google_re_2_f2 = Google_re_2_f[Google_re_2_f['Validity'].isin(['VALID'])]

In [46]:
Google_re_2_f2['Payout_Status_2'] = "Valid"

In [47]:
Google_re_2_f3 = Google_re_2_f2[['sno', 'Payout_Status_2']]

In [48]:
Google_re_2_merge = Google_re_2.merge(Google_re_2_f3, on='sno', how='left')
Google_re_2_merge.drop_duplicates(subset=['sno'], inplace=True)
Google_re_2_merge['New_Payout_Status_2'].fillna(Google_re_2_merge['Payout_Status_2'], inplace=True)
Google_re_2_merge.drop('Payout_Status_2', axis=1, inplace=True)

In [49]:
Google_re_3 = Google_re_2_merge

In [50]:
Google_re_3.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Is in Previous Month Payout Date,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2
0,1,Akola,Akola,7196591.0,8381097116,43260,14549.0,BCR2DN4T2C2KVQ2C,VALID,NaN,...,NaN,NaN,NaN,1,1,No,No,NaN,Google,Valid


# Invalid Analysis and Finding Valid

### Already paid in Match, MN Match

In [51]:
Google_re_3_f = Google_re_3[['sno','Validity', 'Date', 'Is In Source?', 'IngestionStatus', 'Ingestion Date', 'Is in Previous Month Payout', 
'Is in Previous Month Payout Date']]

In [52]:
Google_re_3_f2 = Google_re_3_f[Google_re_3_f['Validity'].isin(['INVALID'])]

In [53]:
Google_re_3_f3 = Google_re_3_f2[Google_re_3_f2['IngestionStatus'].isna()]

In [54]:
Google_re_3_f4 = Google_re_3_f3[Google_re_3_f3['Is In Source?'].isin(['MN Match', "Match"])]

In [55]:
Google_re_3_f4['Payout_Status_1'] = Google_re_3_f4['Is in Previous Month Payout']
Google_re_3_f4['Payout_Status_1'] = Google_re_3_f4['Payout_Status_1'].map({"No Match": np.nan, "Match":"Already Paid"})
Google_re_3_f4['Payout_Status_1'].fillna(Google_re_3_f4['Is In Source?'], inplace=True)
Google_re_3_f4['Payout_Status_1'].replace('MN Match', 'Self OB w/Same Ph.', inplace=True)
Google_re_3_f4['Payout_Status_1'].replace('Match', 'Valid', inplace=True)

In [56]:
Google_re_3_f4_f = Google_re_3_f4[['sno', 'Payout_Status_1']]

In [57]:
Google_re_3_f5 = Google_re_3_f2[~Google_re_3_f2['IngestionStatus'].isna()]

In [58]:
Google_re_3_f5_f = Google_re_3_f5[Google_re_3_f5['Is in Previous Month Payout'].isin(['Match'])]
Google_re_3_f5_f = Google_re_3_f5_f[Google_re_3_f5_f['Is In Source?'].isin(['Match'])]

In [59]:
Google_re_3_f5_f['Is in Previous Month Payout Date'] = pd.to_datetime(Google_re_3_f5_f['Is in Previous Month Payout Date'])
Google_re_3_f5_f['Is in Previous Month Payout Date'] = Google_re_3_f5_f['Is in Previous Month Payout Date'].dt.strftime('%Y/%m/%d')

In [60]:
Google_re_3_f5_f['Ingestion Date'] = pd.to_datetime(Google_re_3_f5_f['Ingestion Date'])
Google_re_3_f5_f['Ingestion Date'] = Google_re_3_f5_f['Ingestion Date'].dt.strftime('%Y/%m/%d')

In [61]:
Google_re_3_f5_f['Payout_Status_2'] = Google_re_3_f5_f['Ingestion Date']>=Google_re_3_f5_f['Is in Previous Month Payout Date']

In [62]:
Google_re_3_f5_f['Payout_Status_2'] = Google_re_3_f5_f['Payout_Status_2'].map({False:'Already Paid', True:'Dynamic_Valid'})

In [63]:
Google_re_3_f5_f2 = Google_re_3_f5_f[['sno', 'Payout_Status_2']]

In [64]:
Google_re_3_f5_f3 = Google_re_3_f5[Google_re_3_f5['Is in Previous Month Payout'].isin(['No Match'])]
Google_re_3_f5_f3 = Google_re_3_f5_f3[Google_re_3_f5_f3['Is In Source?'].isin(['Match'])]

In [65]:
Google_re_3_f5_f3['Payout_Status_3'] = 'Valid'

In [66]:
Google_re_3_f5_f3_f = Google_re_3_f5_f3[['sno', 'Payout_Status_3']]

In [67]:
Google_re_3_f_merge = Google_re_3_f2.merge(Google_re_3_f4_f, on='sno', how='left')
Google_re_3_f_merge.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge2 = Google_re_3_f_merge.merge(Google_re_3_f5_f2, on='sno', how='left')
Google_re_3_f_merge2.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge3 = Google_re_3_f_merge2.merge(Google_re_3_f5_f3_f, on='sno', how='left')
Google_re_3_f_merge3.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'] = np.nan
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_1'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_2'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_3'], inplace=True)

In [68]:
Google_re_3_f_merge3_f = Google_re_3_f_merge3[['sno', 'Is In Source?', 'Temp_Payout_status']]

In [69]:
Google_re_3_f_merge3_f2 = Google_re_3_f_merge3_f[Google_re_3_f_merge3_f['Temp_Payout_status'].isna()]

In [70]:
Google_re_3_f_merge3_f2['Payout_Status_4'] = Google_re_3_f_merge3_f2['Is In Source?']

In [71]:
Google_re_3_f_merge3_f2['Payout_Status_4'].unique()

array(['No Match', 'No Match Dynamic'], dtype=object)

In [72]:
Google_re_3_f_merge3_f2['Payout_Status_4'] = Google_re_3_f_merge3_f2[
    'Payout_Status_4'].map({'No Match':'Invalid_NotOnTargetlist', 'No Match Dynamic':'Dynamic_Expiry'})

In [73]:
Google_re_3_f_merge3_f2_f = Google_re_3_f_merge3_f2[['sno', 'Payout_Status_4']]

In [74]:
Google_re_3_f_merge4 = Google_re_3_f_merge3.merge(Google_re_3_f_merge3_f2_f, on='sno', how='left')
Google_re_3_f_merge4.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge4['Temp_Payout_status'].fillna(Google_re_3_f_merge4['Payout_Status_4'], inplace=True)

In [75]:
Google_re_3_f_merge4_f = Google_re_3_f_merge4[['sno', 'Temp_Payout_status']]

In [76]:
Google_re_3_merge = Google_re_3.merge(Google_re_3_f_merge4_f, on='sno', how='left')
Google_re_3_merge['New_Payout_Status_2'].fillna(Google_re_3_merge['Temp_Payout_status'], inplace=True)
Google_re_3_merge.drop('Temp_Payout_status', axis=1, inplace=True)

In [77]:
Google_re_4 = Google_re_3_merge[['sno','City','TV City',	'Merchant ID',	'StorePhoneNumber',	'Employee Code',	'POD ID',	'MerchantExternalId',	'Validity',
	'InvalidReasons',	'Date',	'Is in RWM File?',	'Is In Source?',	'New_Payout_Status_2',	'Payout Invalid Reason',	'OB Date Check',	'Task Count',	
    'CompletedSubTaskTypes',	'Segment',	'Ingestion Date',	'IngestionStatus',	'Revisit Expire Date',	'Is in Dynamic Source Visit Date Valid?',	
    'Outcome Status',	'Is in Previous Month Payout',	'Is in Previous Month Payout Date',	'Source MID',	'Source MID Match Check',	'Ext_ID Dup?',	'Phone_Dup?',
    'Is in OB CSV file? MID',	'Is in OB CSV file? MN',	'OB Date',	'Data Tag']].rename(columns={'New_Payout_Status_2':'Payout Status'})

In [79]:
Google_re_4.to_csv(output_loc_final+'Final_Google_Reconsilation.csv', index=False)

In [80]:
Google_re_4['Payout Status'].value_counts()

Valid                         158499
Invalid_PhonenumberLookup       5220
Invalid_NotOnTargetlist         2914
Invalid_IncompleteSubTasks      2620
Self OB w/Same Ph.              2000
Already Paid                    1803
Duplicate Ext_ID                1326
Dynamic_Expiry                   239
Dynamic_Valid                    209
Duplicate Phone_No                44
Name: Payout Status, dtype: int64

# RWM File

In [81]:
input_loc_g = input_loc+"RWM/Payout Final/"
#input_loc_g = input_loc+"RWM/Estimated/Consolidated/"

In [82]:
filelist = glob.glob(input_loc_g+"**/*.csv", recursive = True)

In [83]:
filelist

['/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Target/Raw/Nov/RWM/Payout Final/RWM_Payout_status_01_to_16.csv',
 '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout/Target/Raw/Nov/RWM/Payout Final/RWM_Payout_status_17_to_Remaining.csv']

In [84]:
Google_re = pd.DataFrame()

for file in filelist:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
        Google_re = Google_re.append(df)

In [85]:
Google_re.sort_values(by=['sno'], inplace=True)

In [86]:
Google_re['sno'] = Google_re['sno'].astype(str)

In [87]:
Google_re.shape

(190655, 35)

In [88]:
Google_re.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Is in Previous Month Payout Date,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Merchant_ID_Dup,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag
0,1,Pune,Pune,8252706,9579920335,78442,3115.0,BCR2DN4TQD2OZ3BF,VALID,NaN,...,NaN,NaN,NaN,1,1,1,No,No,NaN,RWM


In [89]:
Google_re['Task Count'].replace(np.nan, 0, inplace=True)

In [90]:
len(Google_re['Merchant ID'].unique())

184432

In [91]:
Google_re['Merchant ID'] = Google_re['Merchant ID'].astype(str)
Google_re.sort_values(by=['sno'], inplace=True)
Google_re.sort_values(by=['Validity', 'Merchant_ID_Dup'], ascending=[False, True], inplace=True)
Google_re.drop_duplicates(subset=['Merchant ID'], inplace=True)

In [92]:
len(Google_re['Merchant ID'].unique())

184432

In [93]:
Google_re['Validity'].unique()

array(['VALID', 'INVALID', nan], dtype=object)

# Invalid_IncompleteSubTasks

In [94]:
Google_re_f = Google_re[['sno', 'MerchantExternalId', 'StorePhoneNumber', 'Task Count', 'Validity', 'InvalidReasons']]

In [95]:
Google_re_f2 = Google_re_f[['sno','Validity', 'InvalidReasons' , 'Task Count']]

In [96]:
Google_re_f2 = Google_re_f2[~Google_re_f2['Validity'].isna()]

In [97]:
Google_re_f2['Validity'].replace(np.nan, 0, inplace=True)

In [98]:
Google_re_f3 = Google_re_f2.loc[(Google_re_f2['Task Count']<3)]

In [99]:
Google_re_f3['Payout_Status_1'] = 'Invalid_IncompleteSubTasks'

In [100]:
Google_re_f4 = Google_re_f3[Google_re_f3['InvalidReasons'].str.contains('Invalid_PhonenumberLookup', na=False)]

In [101]:
Google_re_f4['Payout_Status_2'] = 'Invalid_PhonenumberLookup'

In [102]:
Google_re_f4_f = Google_re_f4[['sno', 'Payout_Status_2']]

In [103]:
Google_re_f3_merge = Google_re_f3.merge(Google_re_f4_f, on='sno', how='left')

In [104]:
Google_re_f3_merge.drop_duplicates(subset=['sno'], inplace=True)

In [105]:
Google_re_f3_merge['New_Payout_Status_1'] = np.nan
Google_re_f3_merge['New_Payout_Status_1'].fillna(Google_re_f3_merge['Payout_Status_2'], inplace=True)
Google_re_f3_merge['New_Payout_Status_1'].fillna(Google_re_f3_merge['Payout_Status_1'], inplace=True)

In [106]:
Google_re_f3_merge_f = Google_re_f3_merge[['sno', 'New_Payout_Status_1']]

In [107]:
Google_re_f3_merge_f.head(1)

,sno,New_Payout_Status_1
0,101638,Invalid_IncompleteSubTasks


In [108]:
Google_re_1 = Google_re.merge(Google_re_f3_merge_f, how='left', on='sno')
Google_re_1.drop_duplicates(subset=['sno'], inplace=True)

# Duplicate Merchant External ID

In [109]:
Google_re_1_mid = Google_re_1[['sno', 'Ext_ID Dup?']]
Google_re_1_mn = Google_re_1[['sno', 'Phone_Dup?']]
Google_re_1_merchant_id = Google_re_1[['sno', 'Merchant_ID_Dup']]

In [110]:
Google_re_1_mid_f = Google_re_1_mid.loc[(Google_re_1_mid['Ext_ID Dup?']>1)]

In [111]:
Google_re_1_mid_f['Payout_Status_1'] = "Duplicate Ext_ID"

In [112]:
Google_re_1_mid_f2 = Google_re_1_mid_f[['sno', 'Payout_Status_1']]

In [113]:
Google_re_1_mn_f = Google_re_1_mn.loc[(Google_re_1_mn['Phone_Dup?']>1)]

In [114]:
Google_re_1_mn_f['Payout_Status_2'] = "Duplicate Phone_No"

In [115]:
Google_re_1_mn_f2 = Google_re_1_mn_f[['sno', 'Payout_Status_2']]

In [116]:
Google_re_1_merchant_id_f = Google_re_1_merchant_id.loc[(Google_re_1_merchant_id['Merchant_ID_Dup']>1)]

In [117]:
Google_re_1_merchant_id_f['Payout_Status_3'] = "Duplicate Merchant_ID"

In [118]:
Google_re_1_merchant_id_f2 = Google_re_1_merchant_id_f[['sno', 'Payout_Status_3']]

In [119]:
Google_re_1_merge = Google_re_1.merge(Google_re_1_mid_f2, on='sno', how='left')
Google_re_1_merge.drop_duplicates(subset=['sno'], inplace=True)

In [120]:
Google_re_1_merge2 = Google_re_1_merge.merge(Google_re_1_mn_f2, on='sno', how='left')
Google_re_1_merge2.drop_duplicates(subset=['sno'], inplace=True)

In [121]:
Google_re_1_merge2 = Google_re_1_merge2.merge(Google_re_1_merchant_id_f2, on='sno', how='left')
Google_re_1_merge2.drop_duplicates(subset=['sno'], inplace=True)

In [122]:
Google_re_2 = Google_re_1_merge2

In [123]:
Google_re_2['New_Payout_Status_2'] = np.nan
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['Payout_Status_1'], inplace=True)
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['Payout_Status_2'], inplace=True)
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['Payout_Status_3'], inplace=True)
Google_re_2['New_Payout_Status_2'].fillna(Google_re_2['New_Payout_Status_1'], inplace=True)
Google_re_2.drop(['Payout_Status_1', 'Payout_Status_2', 'Payout_Status_3','New_Payout_Status_1'], axis=1, inplace=True)

In [124]:
Google_re_2.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Merchant_ID_Dup,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2
0,1,Pune,Pune,8252706,9579920335,78442,3115.0,BCR2DN4TQD2OZ3BF,VALID,NaN,...,NaN,NaN,1,1,1,No,No,NaN,RWM,NaN


# Valid

In [125]:
Google_re_2_f = Google_re_2[['sno', 'Validity']]

In [126]:
Google_re_2_f['Validity'].value_counts()

VALID      159297
INVALID      8517
Name: Validity, dtype: int64

In [127]:
Google_re_2_f2 = Google_re_2_f[Google_re_2_f['Validity'].isin(['VALID'])]

In [128]:
Google_re_2_f2['Payout_Status_2'] = "Valid"

In [129]:
Google_re_2_f3 = Google_re_2_f2[['sno', 'Payout_Status_2']]

In [130]:
Google_re_2_merge = Google_re_2.merge(Google_re_2_f3, on='sno', how='left')
Google_re_2_merge.drop_duplicates(subset=['sno'], inplace=True)
Google_re_2_merge['New_Payout_Status_2'].fillna(Google_re_2_merge['Payout_Status_2'], inplace=True)
Google_re_2_merge.drop('Payout_Status_2', axis=1, inplace=True)

In [131]:
Google_re_3 = Google_re_2_merge

In [132]:
Google_re_3.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Merchant_ID_Dup,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2
0,1,Pune,Pune,8252706,9579920335,78442,3115.0,BCR2DN4TQD2OZ3BF,VALID,NaN,...,NaN,NaN,1,1,1,No,No,NaN,RWM,Valid


# Invalid Analysis and Finding Valid

### Already paid in Match, MN Match

In [133]:
Google_re_3_f = Google_re_3[['sno','Validity', 'Date', 'Is In Source?', 'IngestionStatus', 'Ingestion Date', 'Is in Previous Month Payout', 
'Is in Previous Month Payout Date']]

In [134]:
Google_re_3_f2 = Google_re_3_f[Google_re_3_f['Validity'].isin(['INVALID'])]

In [135]:
Google_re_3_f3 = Google_re_3_f2[Google_re_3_f2['IngestionStatus'].isna()]

In [136]:
Google_re_3_f4 = Google_re_3_f3[Google_re_3_f3['Is In Source?'].isin(['MN Match', "Match"])]

In [137]:
Google_re_3_f4['Payout_Status_1'] = Google_re_3_f4['Is in Previous Month Payout']
Google_re_3_f4['Payout_Status_1'] = Google_re_3_f4['Payout_Status_1'].map({"No Match": np.nan, "Match":"Already Paid"})
Google_re_3_f4['Payout_Status_1'].fillna(Google_re_3_f4['Is In Source?'], inplace=True)
Google_re_3_f4['Payout_Status_1'].replace('MN Match', 'Self OB w/Same Ph.', inplace=True)
Google_re_3_f4['Payout_Status_1'].replace('Match', 'Valid', inplace=True)

In [138]:
Google_re_3_f4_f = Google_re_3_f4[['sno', 'Payout_Status_1']]

In [139]:
Google_re_3_f5 = Google_re_3_f2[~Google_re_3_f2['IngestionStatus'].isna()]

In [140]:
Google_re_3_f5_f = Google_re_3_f5[Google_re_3_f5['Is in Previous Month Payout'].isin(['Match'])]
Google_re_3_f5_f = Google_re_3_f5_f[Google_re_3_f5_f['Is In Source?'].isin(['Match'])]

In [141]:
Google_re_3_f5_f['Is in Previous Month Payout Date'] = pd.to_datetime(Google_re_3_f5_f['Is in Previous Month Payout Date'])
Google_re_3_f5_f['Is in Previous Month Payout Date'] = Google_re_3_f5_f['Is in Previous Month Payout Date'].dt.strftime('%Y/%m/%d')

In [142]:
Google_re_3_f5_f['Ingestion Date'] = pd.to_datetime(Google_re_3_f5_f['Ingestion Date'])
Google_re_3_f5_f['Ingestion Date'] = Google_re_3_f5_f['Ingestion Date'].dt.strftime('%Y/%m/%d')

In [143]:
Google_re_3_f5_f['Payout_Status_2'] = Google_re_3_f5_f['Ingestion Date']>=Google_re_3_f5_f['Is in Previous Month Payout Date']

In [144]:
Google_re_3_f5_f['Payout_Status_2'] = Google_re_3_f5_f['Payout_Status_2'].map({False:'Already Paid', True:'Dynamic_Valid'})

In [145]:
Google_re_3_f5_f2 = Google_re_3_f5_f[['sno', 'Payout_Status_2']]

In [146]:
Google_re_3_f5_f3 = Google_re_3_f5[Google_re_3_f5['Is in Previous Month Payout'].isin(['No Match'])]
Google_re_3_f5_f3 = Google_re_3_f5_f3[Google_re_3_f5_f3['Is In Source?'].isin(['Match'])]

In [147]:
Google_re_3_f5_f3['Payout_Status_3'] = 'Valid'

In [148]:
Google_re_3_f5_f3_f = Google_re_3_f5_f3[['sno', 'Payout_Status_3']]

In [149]:
Google_re_3_f_merge = Google_re_3_f2.merge(Google_re_3_f4_f, on='sno', how='left')
Google_re_3_f_merge.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge2 = Google_re_3_f_merge.merge(Google_re_3_f5_f2, on='sno', how='left')
Google_re_3_f_merge2.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge3 = Google_re_3_f_merge2.merge(Google_re_3_f5_f3_f, on='sno', how='left')
Google_re_3_f_merge3.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'] = np.nan
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_1'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_2'], inplace=True)
Google_re_3_f_merge3['Temp_Payout_status'].fillna(Google_re_3_f_merge3['Payout_Status_3'], inplace=True)

In [150]:
Google_re_3_f_merge3_f = Google_re_3_f_merge3[['sno', 'Is In Source?', 'Temp_Payout_status']]

In [151]:
Google_re_3_f_merge3_f2 = Google_re_3_f_merge3_f[Google_re_3_f_merge3_f['Temp_Payout_status'].isna()]

In [152]:
Google_re_3_f_merge3_f2['Payout_Status_4'] = Google_re_3_f_merge3_f2['Is In Source?']

In [153]:
Google_re_3_f_merge3_f2['Payout_Status_4'].unique()

array(['No Match', 'No Match Dynamic'], dtype=object)

In [154]:
Google_re_3_f_merge3_f2['Payout_Status_4'] = Google_re_3_f_merge3_f2[
    'Payout_Status_4'].map({'No Match':'Invalid_NotOnTargetlist', 'No Match Dynamic':'Dynamic_Expiry'})

In [155]:
Google_re_3_f_merge3_f2_f = Google_re_3_f_merge3_f2[['sno', 'Payout_Status_4']]

In [156]:
Google_re_3_f_merge4 = Google_re_3_f_merge3.merge(Google_re_3_f_merge3_f2_f, on='sno', how='left')
Google_re_3_f_merge4.drop_duplicates(subset=['sno'], inplace=True)
Google_re_3_f_merge4['Temp_Payout_status'].fillna(Google_re_3_f_merge4['Payout_Status_4'], inplace=True)

In [157]:
Google_re_3_f_merge4_f = Google_re_3_f_merge4[['sno', 'Temp_Payout_status']]

In [158]:
Google_re_3_merge = Google_re_3.merge(Google_re_3_f_merge4_f, on='sno', how='left')
Google_re_3_merge['New_Payout_Status_2'].fillna(Google_re_3_merge['Temp_Payout_status'], inplace=True)
Google_re_3_merge.drop('Temp_Payout_status', axis=1, inplace=True)

In [159]:
rwm_re = Google_re_3_merge

# Not Found Cases Analysis

In [160]:
rwm_re.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Source MID,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Merchant_ID_Dup,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2
0,1,Pune,Pune,8252706,9579920335,78442,3115.0,BCR2DN4TQD2OZ3BF,VALID,NaN,...,NaN,NaN,1,1,1,No,No,NaN,RWM,Valid


In [161]:
rwm_re_f = rwm_re[rwm_re['Is in Google Csv File?'].isin(["No"])]

In [162]:
rwm_re_f2 = rwm_re_f[['sno', 'Is In Source?', 'Segment', 'Is in OB CSV file? MID', 'Is in OB CSV file? MN', 'OB Date', 'OB Date Check']]

### MN Match

In [163]:
rwm_re_f2_f = rwm_re_f2[rwm_re_f2['Is In Source?'].isin(['MN Match'])]

In [164]:
rwm_re_f2_f['Payout_Status_1'] = rwm_re_f2_f['OB Date Check']
rwm_re_f2_f['Payout_Status_1'] = rwm_re_f2_f['Payout_Status_1'].map({True:'Re-onboarded', False:'Self OB w/Same Ph.', np.nan:'Self OB w/Same Ph.'})

In [165]:
rwm_re_f2_f2 = rwm_re_f2_f[['sno', 'Payout_Status_1']]

### Match

In [166]:
rwm_re_f2_f3 = rwm_re_f2[rwm_re_f2['Is In Source?'].isin(['Match'])]

In [167]:
rwm_re_f2_f3['Payout_Status_2'] = rwm_re_f2_f3['OB Date Check']
rwm_re_f2_f3['Payout_Status_2'] = rwm_re_f2_f3['Payout_Status_2'].map({True:'Re-onboarded', False:'Not Found', np.nan:'Not Found'})

In [168]:
rwm_re_f2_f4 = rwm_re_f2_f3[['sno', 'Payout_Status_2']]

### No Match

In [169]:
rwm_re_f2_f5 = rwm_re_f2[rwm_re_f2['Is In Source?'].isin(['No Match'])]

In [170]:
rwm_re_f2_f5['Payout_Status_3'] = rwm_re_f2_f5['OB Date Check']
rwm_re_f2_f5['Payout_Status_3'] = rwm_re_f2_f5['Payout_Status_3'].map({True:'Invalid_NotOnTargetlist', False:'Not Found', np.nan:'Not Found'})

In [171]:
rwm_re_f2_f6 = rwm_re_f2_f5[['sno', 'Payout_Status_3']]

### No Match Dynamic

In [172]:
rwm_re_f2_f7 = rwm_re_f2[rwm_re_f2['Is In Source?'].isin(['No Match Dynamic'])]
rwm_re_f2_f7['Payout_Status_4'] = rwm_re_f2_f7['OB Date Check']
rwm_re_f2_f7['Payout_Status_4'] = rwm_re_f2_f7['Payout_Status_4'].map({True:'Not Found', False:'Not Found', np.nan:'Not Found'})

In [173]:
rwm_re_f2_f8 = rwm_re_f2_f7[['sno', 'Payout_Status_4']]

### Merging the above criteria in Not Found

In [174]:
rwm_re_f2_merge = rwm_re_f2.merge(rwm_re_f2_f2, on='sno', how='left')
rwm_re_f2_merge.drop_duplicates(subset=['sno'], inplace=True)
rwm_re_f2_merge2 = rwm_re_f2_merge.merge(rwm_re_f2_f4, on='sno', how='left')
rwm_re_f2_merge2.drop_duplicates(subset=['sno'], inplace=True)
rwm_re_f2_merge3 = rwm_re_f2_merge2.merge(rwm_re_f2_f6, on='sno', how='left')
rwm_re_f2_merge3.drop_duplicates(subset=['sno'], inplace=True)
rwm_re_f2_merge4 = rwm_re_f2_merge3.merge(rwm_re_f2_f8, on='sno', how='left')
rwm_re_f2_merge4.drop_duplicates(subset=['sno'], inplace=True)

In [175]:
rwm_re_f2_merge4['Temp_Payout_Status_2'] = rwm_re_f2_merge4['Payout_Status_1']
rwm_re_f2_merge4['Temp_Payout_Status_2'].fillna(rwm_re_f2_merge4['Payout_Status_2'], inplace=True)
rwm_re_f2_merge4['Temp_Payout_Status_2'].fillna(rwm_re_f2_merge4['Payout_Status_3'], inplace=True)
rwm_re_f2_merge4['Temp_Payout_Status_2'].fillna(rwm_re_f2_merge4['Payout_Status_4'], inplace=True)

In [176]:
rwm_re_f2_merge4_f = rwm_re_f2_merge4[['sno', 'Temp_Payout_Status_2']]

In [177]:
rwm_re_merge = rwm_re.merge(rwm_re_f2_merge4_f, on='sno', how='left')
rwm_re_merge.drop_duplicates(subset=['sno'], inplace=True)

In [178]:
rwm_re_merge.head(1)

,sno,City,TV City,Merchant ID,StorePhoneNumber,Employee Code,POD ID,MerchantExternalId,Validity,InvalidReasons,...,Source MID Match Check,Ext_ID Dup?,Phone_Dup?,Merchant_ID_Dup,Is in OB CSV file? MID,Is in OB CSV file? MN,OB Date,Data Tag,New_Payout_Status_2,Temp_Payout_Status_2
0,1,Pune,Pune,8252706,9579920335,78442,3115.0,BCR2DN4TQD2OZ3BF,VALID,NaN,...,NaN,1,1,1,No,No,NaN,RWM,Valid,NaN


In [179]:
rwm_re_merge['New_Payout_Status_2'].fillna(rwm_re_merge['Temp_Payout_Status_2'], inplace=True)
rwm_re_merge.drop('Temp_Payout_Status_2', axis=1, inplace=True)

In [180]:
rwm_re_merge_2 = rwm_re_merge[['sno','City',	'TV City',	'Merchant ID',	'StorePhoneNumber',	'Employee Code',	'POD ID',	'MerchantExternalId',	'Validity',
	'InvalidReasons',	'Date',	'Is in Google Csv File?',	'Is In Source?',	'New_Payout_Status_2',	'Payout Invalid Reason',	'OB Date Check',	'Task Count',	
    'CompletedSubTaskTypes',	'Segment',	'Ingestion Date',	'IngestionStatus',	'Revisit Expire Date',	'Is in Dynamic Source Visit Date Valid?',	
    'Outcome Status',	'Is in Previous Month Payout',	'Is in Previous Month Payout Date',	'Source MID',	'Source MID Match Check',	'Ext_ID Dup?',	
    'Phone_Dup?', "Merchant_ID_Dup" ,'Is in OB CSV file? MID',	'Is in OB CSV file? MN',	'OB Date',	'Data Tag']].rename(
        columns={'New_Payout_Status_2':'Payout Status'})

In [181]:
rwm_re_merge_2.to_csv(output_loc_final+'Payout_Consolidate.csv', index=False)

In [182]:
rwm_re_merge_2['Payout Status'].value_counts()

Valid                         160571
Re-onboarded                    8112
Not Found                       4985
Invalid_NotOnTargetlist         3468
Self OB w/Same Ph.              2516
Invalid_PhonenumberLookup       1991
Invalid_IncompleteSubTasks      1336
Already Paid                     793
Duplicate Ext_ID                 408
Dynamic_Valid                    208
Dynamic_Expiry                    40
Duplicate Phone_No                 4
Name: Payout Status, dtype: int64

In [183]:
len(rwm_re_merge_2)

184432